In [1]:
import pandas as pd
import numpy as np

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error

In [2]:
path = 'C:/Users/user/Desktop/PLoSONE/data/'

data = pd.read_csv(path + 'dataset_household_PLoS ONE.csv')
data.head()

,Date,Hour_x,Hour_y,DOTW_x,DOTW_y,Holi,Temp,Humi,WS,THI,WCT,Cons_1,Holi_1,Cons_7,Holi_7,Cons_avg,Consumption
0,2016-01-11 18:00,-1.000000,-1.840000e-16,0.781831,0.62349,0,5.9,92.0,5.0,43.29672,5.103854,0,0,0,0,0.0,330
1,2016-01-11 19:00,-0.965926,2.588190e-01,0.781831,0.62349,0,6.0,91.0,6.0,43.55240,4.872989,0,0,0,0,0.0,1430
2,2016-01-11 20:00,-0.866025,5.000000e-01,0.781831,0.62349,0,6.0,88.0,6.0,43.80320,4.872989,0,0,0,0,0.0,690
3,2016-01-11 21:00,-0.707107,7.071068e-01,0.781831,0.62349,0,6.0,87.0,6.0,43.88680,4.872989,0,0,0,0,0.0,780
4,2016-01-11 22:00,-0.500000,8.660254e-01,0.781831,0.62349,0,5.6,88.0,6.0,43.13072,4.413133,0,0,0,0,0.0,620


In [3]:
data_drop = data.drop(columns = 'Date', axis = 1)
data_drop.head()

,Hour_x,Hour_y,DOTW_x,DOTW_y,Holi,Temp,Humi,WS,THI,WCT,Cons_1,Holi_1,Cons_7,Holi_7,Cons_avg,Consumption
0,-1.000000,-1.840000e-16,0.781831,0.62349,0,5.9,92.0,5.0,43.29672,5.103854,0,0,0,0,0.0,330
1,-0.965926,2.588190e-01,0.781831,0.62349,0,6.0,91.0,6.0,43.55240,4.872989,0,0,0,0,0.0,1430
2,-0.866025,5.000000e-01,0.781831,0.62349,0,6.0,88.0,6.0,43.80320,4.872989,0,0,0,0,0.0,690
3,-0.707107,7.071068e-01,0.781831,0.62349,0,6.0,87.0,6.0,43.88680,4.872989,0,0,0,0,0.0,780
4,-0.500000,8.660254e-01,0.781831,0.62349,0,5.6,88.0,6.0,43.13072,4.413133,0,0,0,0,0.0,620


In [4]:
x_train = data_drop.iloc[:2310]
x_test = data_drop.iloc[2310:]
y_train = data_drop.iloc[:2310]

In [5]:
# target 정의 
y = y_train['Consumption']

In [6]:
# 데이터 셋 분리
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y, test_size=0.3, random_state=42)

In [8]:
# LightGBM
model = LGBMRegressor()

params = {'n_estimators': [1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [64],
    'subsample': [0.5],
    'colsample_bytree': [1.0],
    'boosting_type': ['gbdt', 'dart']
}

# GridSEarch
gr = GridSearchCV(model, param_grid = params, cv = 5)
gr.fit(X_train, Y_train)

print(gr.best_params_)

{'boosting_type': 'gbdt', 'colsample_bytree': 1.0, 'learning_rate': 0.01, 'n_estimators': 1000, 'num_leaves': 64, 'subsample': 0.5}


In [9]:
lgb = LGBMRegressor(random_state=42, boosting_type='gbdt',colsample_bytree=1.0, learning_rate=0.01, 
                   n_estimators=1000, num_leaves=64, subsample=0.5) # 다시 넣기
lgb.fit(X_train, Y_train)

y_pred = lgb.predict(X_valid)
y_pred

array([ 330.00567665,  370.01027559,  539.58749555,  222.14565606,
        559.26715769,  279.99050865, 1111.16579171,  330.01587452,
        380.06619306,  320.02212464,  449.81803363,  807.65685707,
        490.27586322,  409.75016215,  430.58044475,  680.06357861,
        820.54538936,  779.97286738,  909.53073102,  690.37201469,
       1008.31381661,  280.00703421,  290.03775739,  359.12778344,
       2068.83389243,  609.98329721,  520.06762901,  350.72144766,
        269.94036514,  808.23279228,  300.02215568,  280.01626128,
        279.88585512,  310.00350156,  359.99758112,  390.06069903,
        329.8915047 ,  279.92350018,  219.74088595,  439.85838525,
       2080.6065295 ,  319.62473015,  530.35067265,  279.97115565,
        346.52355862,  280.06060859,  436.23573469,  270.13874685,
        389.99375531,  208.76225166,  280.05384789,  957.22560617,
        350.05345682,  279.99813701,  329.99214295,  380.07593385,
        609.89709537,  330.01361524,  699.92109324,  250.10774

In [10]:
# 실제값 
y_predict = lgb.predict(x_test)
y_predict.shape

(979,)

In [30]:
y_predict

array([ 339,  300,  329,  300,  339,  289,  300,  329,  350,  890,  679,
       2742,  790,  639,  420,  450,  599,  978, 1470,  699,  729,  550,
        420,  319,  339,  339,  409,  319,  250,  319,  310,  330,  289,
        330,  300,  289,  490,  310,  299,  250,  349,  440, 1763,  650,
        710,  730,  289,  269,  310,  320,  230,  310,  319,  310,  679,
        459,  780,  310,  300,  240,  310,  309,  249,  319,  289,  340,
       1610,  698,  680,  499,  329,  310,  229,  329,  310,  240,  310,
        319,  280,  420,  330,  350,  350,  389,  249,  309,  350, 1045,
        465,  425, 1367, 1433,  710,  629,  409,  319,  229,  319,  329,
        300,  240,  320,  420,  330,  280,  320,  310,  300,  329,  315,
        385,  365,  435,  494, 1257, 1113,  714, 1027,  427,  399,  419,
        349,  389,  409,  389,  409,  470,  369, 1008, 1650, 1891, 2356,
       1555, 2230, 1435,  500, 2169, 1310, 2783,  849,  609,  640,  379,
        340,  320,  339,  300,  340,  300,  340,  3

In [11]:
df = pd.DataFrame({'Date' : data['Date'].iloc[2310:], 'Actual' : data['Consumption'].iloc[2310:], 'Predict' : y_predict}).reset_index(drop=True)
df.to_csv('C:/Users/user/Desktop/PLoSONE/predict/household_PLoS_LightGBM.csv', header=True, index=False)